In [11]:
from gurobipy import *
# Model data
generators, minGen, maxGen, a, b = multidict({
'gen1': [20.0, 400.0, 20.0, 0.05],
'gen2': [20.0, 300.0, 25.0, 0.10],
'gen3': [30.0, 250.0, 40.0, 0.20] })

buses, loads = multidict({
'bus1': 0.0,
'bus2': 0.0,
'bus3': 200.0,
'bus4': 400.0})

MaxFlow = 300.0
MinFlow = -300.0
edges, minFlow, maxFlow, reactance = multidict({
('Bus1', 'Bus2'): [MinFlow, MaxFlow, 0.007],
('Bus2', 'Bus1'): [MinFlow, MaxFlow, 0.007],
('Bus2', 'Bus3'): [MinFlow, MaxFlow, 0.01],
('Bus3', 'Bus2'): [MinFlow, MaxFlow, 0.01],
('Bus3', 'Bus4'): [MinFlow, MaxFlow, 0.006],
('Bus4', 'Bus3'): [MinFlow, MaxFlow, 0.006],
('Bus4', 'Bus1'): [MinFlow, MaxFlow, 0.05],
('Bus1', 'Bus4'): [MinFlow, MaxFlow, 0.05]})

In [13]:
print(b)

{'gen2': 0.1, 'gen3': 0.2, 'gen1': 0.05}


In [ ]:
#parameters
model.a = Param(model.G, initialize={1:20.0 , 2:25.0 , 3:40.0}, 
                doc='Parameter a')
model.b = Param(model.G, initialize={1:0.05 , 2:0.10 , 3:0.20}, 
                doc='Parameter b')
#lone parameter, constant
model.D = Param(initialize=600.0, doc='Load')

In [ ]:
#variables
model.P = Var(model.G, doc='Generation per Generator')
model.Angles = Var(model.Bus, doc='Angles of Buses')

In [ ]:
#constraints
def generation_r(model,i):
    return (model.Pmin[i] , model.P[i] , model.Pmax[i])
model.GenCon = Constraint(model.G, rule = generation_r)

def demand_r(model):
    return model.D == sum(model.P[i] for i in model.G)
model.DemandCon = Constraint(model.G, rule = demand_r)

def slack_r(model):
    return model.Angles[1] == 0
model.SlackBusCon = Constraint(rule=slack_r)

def bus_r(model,i):
    if i == 4:
        return(-model.Loads[i] == 
              ((model.BaseP/model.X[i,i-1])*
               (model.Angles[i]-model.Angles[i-1]) + 
               (model.BaseP/model.X[i,i-3])*
               (model.Angles[i]-model.Angles[i-3])))
    elif i == 1:
        return((model.P[i] - model.Loads[i]) == 
              ((model.BaseP/model.X[i,i+1])*
               (model.Angles[i]-model.Angles[i+1]) + 
               (model.BaseP/model.X[i,i+3])*
               (model.Angles[i]-model.Angles[i+3])))    
    else:
        return((model.P[i] - model.Loads[i]) ==     
              ((model.BaseP/model.X[i,i+1])*
               (model.Angles[i]-model.Angles[i+1]) +
               (model.BaseP/model.X[i,i-1])*
               (model.Angles[i]-model.Angles[i-1])))
model.BusCon = Constraint(model.Bus, rule=bus_r)

def flow_r(model,i,j):
    if i == j:
        return Constraint.Skip
    return (model.PFmin , 
            (model.BaseP/model.X[i,j])*(model.Angles[i]-model.Angles[j]) , 
            model.PFmax)
model.FlowCon = Constraint(model.Bus, model.Bus, rule=flow_r)

In [ ]:
#objective
def cost_rule(model):
    return sum(model.a[i]*model.P[i] + 
               0.5*model.b[i]*model.P[i]**2
              for i in model.G)
#default is to minimize        
model.OBJ = Objective(rule=cost_rule) 

In [ ]:
if solverexe == "gurobi":
    solver = SolverFactory(solverexe)   
else:
    solver = SolverFactory(solverexe, 
                           executable=os.path.join(dirsolver, solverexe))
instance = model.create_instance()
instance.dual = Suffix(direction=Suffix.IMPORT)
results = solver.solve(instance)

In [ ]:
instance.display()

In [ ]:
instance.pprint()

In [ ]:
print ("Total cost: ", "{:,}".format(round(value(instance.OBJ),2)))

In [ ]:
print ("Duals")
from pyomo.core import Constraint
for c in instance.component_objects(Constraint, active=True):
    print ("   Constraint",c)
    cobject = getattr(instance, str(c))
    for index in cobject:
        print ("      ", index, instance.dual[cobject[index]])